In [1]:
import pandas as pd
import numpy as np
import apyori

In [2]:
#Предобработка 

# !pip install kaggle
from google.colab import files
files.upload()#upload kaggle.json
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/#Make directory named kaggle and copy kaggle.json file there.
! chmod 600 ~/.kaggle/kaggle.json#Change the permissions of the file.
! kaggle datasets list#check
#download dataset to content folder in google collab
!kaggle datasets download -d netflix-inc/netflix-prize-data
!unzip -q netflix-prize-data.zip
data_all = []
for i in range(4):
    data = pd.read_csv(r'/content/combined_data_' + str(i+1)+'.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
    data_all.append(data)

df = pd.concat(data_all)
df.head()

# data_all = []
# for i in range(4):
#     data = pd.read_csv('/combined_data_' + str(i+1)+'.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
#     data_all.append(data)
# df = pd.concat(data_all)
df.index = np.arange(0,len(df))
print('Full dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])

df_nan = pd.DataFrame(pd.isnull(df.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,np.abs(i-j-1)), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

df = df[pd.notnull(df['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)
print('-Dataset examples-')
print(df.sample(3))

df.to_csv('/srv/aprosvetov/netflix/data_prep.csv', sep=';', index = None)

In [185]:
# Готовый набор
df = pd.read_csv(r'data_prep.csv', sep=';')
print('Уникальных фильмов:',df.Cust_Id.nunique())
print('Уникальных юзеров:',df.Movie_Id.nunique())
df.head()

Уникальных фильмов: 480189
Уникальных юзеров: 17770


,Cust_Id,Rating,Movie_Id
0,1488844,3.0,1
1,822109,5.0,1
2,885013,4.0,1
3,30878,4.0,1
4,823519,3.0,1


Из полученного датасета возьмем только те записи, у которых наивысший рейтинг (5) и объединим их по "Cust_Id". Фильмы сгруппируем в строчку с разделителем "пробел" так, чтобы для каждого пользователя была строка с Id тех фильмов, которые ему понравились

In [5]:
good = df[df['Rating']==5].groupby('Cust_Id')['Movie_Id'].apply(lambda r: ' '.join([str(A) for A in r]))
good.head()

Cust_Id
6     175 457 886 1467 2372 2452 2782 3290 4043 4633...
7     8 30 83 175 257 283 285 313 357 457 458 468 50...
8     1202 1799 1905 2186 3610 3925 4306 5054 5317 5...
10    473 985 1542 1905 2172 3124 3371 3962 4043 430...
25                4432 6786 7605 9326 10643 15107 15270
Name: Movie_Id, dtype: object

ID пользователя, дальше через пробел фильмы, которые нравятся этому пользователю

Мы можем регулировать их количество, меняя параметры алгоритмов. Посмотрим, какие ассоциативные правила получаются для support = 0.04

In [6]:
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.04, 
                                   min_confidence=0.1, min_lift=2, 
                                   min_length=2)
association_rules

<generator object apriori at 0x00000195B96FC7B0>

In [7]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]
asr_df

,from,to,confidence,support,lift
0,10042,10820,0.266215,0.0415732,4.11022
1,10042,10947,0.351478,0.0548881,2.42194
2,10042,11064,0.348135,0.0543661,2.3007
3,10042,11089,0.331616,0.0517864,2.48827
4,10042,11283,0.454613,0.0709941,2.20623
...,...,...,...,...,...
2018,11521,5582 16265 7057 2452 14961 7230,0.199233,0.0414848,4.63066
2019,11521,9628 5582 7057 2452 14961 7230,0.195898,0.0407902,4.63469
2020,14240,5582 16265 7057 2452 14961 7230,0.208372,0.0417112,4.84307
2021,14240,9628 5582 7057 2452 14961 7230,0.204569,0.0409498,4.83983


Мы видим здесь таблицу, где 2023 ассоциативных правила, и для каждого рассчитаны известные нам показатели.

In [8]:
asr_df.sort_values('lift').tail(10)

,from,to,confidence,support,lift
1881,14961,2452 16265 7057 7230,0.375413,0.046806,7.36834
2000,14961,9628 5582 16265 7057 7230,0.330554,0.041213,7.38021
1999,14961,5582 16265 7057 2452 7230,0.345086,0.0430248,7.39518
325,14454,457,0.631952,0.0607572,7.40661
1187,7057,9628 7230,0.449937,0.0562664,7.43479
143,11443,17627,0.734164,0.0657225,7.95834
768,11443,17627 2452,0.463653,0.0415063,8.6031
764,11443,11521 17627,0.465292,0.041653,8.75651
766,11443,17627 12338,0.490663,0.0439243,9.47831
313,14302,16147,0.741323,0.0403588,12.7193


In [119]:
asr_df.iloc[315]

from              14313
to                16377
confidence     0.460884
support       0.0607399
lift            2.30095
Name: 315, dtype: object

Для того чтобы перейти от Id фильмов, к их названиям, нужно загрузить еще один файл, в котором содержится Id фильма, год его производства и название:

In [53]:
titles = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", 
                     header = None, 
                     names = ['Movie_Id', 'Year', 'Name'])
titles.head()

,Movie_Id,Year,Name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW


In [97]:
def get_rule_title(rule):
    """процедура, которая выводит названия фильмов в ассоциативном правиле и фильм, 
       которое это ассоциативное правило рекомендует:"""
    print(titles[titles.Movie_Id.isin(rule['from'].split(' '))]['Name'].values)
    print('----------->')
    print(titles[titles.Movie_Id.isin([int(i) for i in rule['to'].split(' ')])]['Name'].values)

Можем посмотреть, как выглядит это правило. Например, вызовем сотое правило:

In [107]:
get_rule_title(asr_df.loc[99])

['Monsters']
----------->
['Finding Nemo (Widescreen)']


То есть, если человеку нравится фильм «Монстры», то мы советуем ему посмотреть фильм «В поисках Немо».

In [120]:
get_rule_title(asr_df.loc[315])

['The Patriot']
----------->
['The Green Mile']


In [101]:
get_rule_title(asr_df.sample(1).iloc[0])

['Lord of the Rings: The Two Towers']
----------->
['The Lord of the Rings: The Fellowship of the Ring: Extended Edition'
 'Lord of the Rings: The Return of the King: Extended Edition']


In [102]:
asr_df.sample(1)

,from,to,confidence,support,lift
277,13728,8904,0.27303,0.0409714,2.62824


Перейдём к построению рекомендаций для случайного человека под id=159992. Посмотрим, какие фильмы он смотрел и как он их оценил. 

In [131]:
j = 159992

In [132]:
titles[titles.Movie_Id.isin(good.iloc[j].split(' '))]['Name']

1144     The Wedding Planner
2151         What Women Want
5316       Miss Congeniality
6286            Pretty Woman
6385              Sister Act
7233            Men of Honor
10358          Runaway Bride
11148      Maid in Manhattan
13049       Two Weeks Notice
15581     Sweet Home Alabama
Name: Name, dtype: object

In [133]:
# можем пройтись по всем правилам в нашей таблице и проверить, если они присутствуют в просмотрах человека и у них 
#высокий рейтинг, значит это правило ему подходит и мы можем добавить этот фильм в список рекомендаций.
def print_rule_title(rule):
    return (titles[titles.Movie_Id == int(rule['to'])]['Name'].values)
    

result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(' ')) & set(asr_df['from'].loc[A].split(' '))) == len(asr_df['from'].loc[A].split(' ')):
        result.append(print_rule_title(asr_df.loc[A])[0])
print(set(result))

{'Dirty Dancing', 'Pretty Woman'}


Если вы запустите этот код, то увидите, что здесь в рекомендации будут два фильма:

'Pretty Woman', 'Dirty Dancing'

Один из этих фильмов человек уже видел.

Сложность такой рекомендательной системы состоит в том, что мы ограничены теми ассоциативными правилами, которые были созданы. Если фильм редкий, то он в эти ассоциативные правила, скорее всего, не попадёт.

Также для разных людей будет разное количество рекомендаций, в зависимости от того, сколько фильмов посмотрел каждый из них. И, если нам захочется порекомендовать каждому пользователю минимум три фильма, то не с каждым пользователем мы сможем это сделать.

Эту проблему можно решить с помощью алгоритма Коллаборативная фильтрация. 

In [134]:
# Возьмём подвыборку из 10000 случайных кастомеров и 5000 фильмов. 
cust_sample = df.Cust_Id.sample(10000)
movie_sample = df.Movie_Id.sample(5000)


Для генерации простых рекомендаций с помощью коллаборативной фильтрации можно воспользоваться модулем surprise. Загрузим в модуль surprise наш датасет с помощью метода Reader. 

In [137]:
# !pip install surprise
# conda install -c conda-forge scikit-surprise

In [3]:
import surprise
from surprise import Reader, Dataset

In [138]:
#Возьмем только те оценки, которые относятся к выбранному подмножеству кастомеров и только те оценки, 
# которые относятся к выбранному подмножеству фильмов. Именно в такой последовательности — сначала Cust_Id, 
# затем Movie_Id, затем Rating. 

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[df.Cust_Id.isin(cust_sample) &
                              df.Movie_Id.isin(movie_sample)][['Cust_Id', 'Movie_Id', 'Rating']], reader)

В модуле surprise есть несколько реализаций коллаборативной фильтрации. Мы возьмем одну из самых самых простых — принцип ближайших соседей.

Принцип коллаборативной фильтрации заключается в следующем:

Для каждого человека находится небольшое множество похожих на него зрителей с оценками примерно такими же, какие поставил человек на ряд фильмов (item). Из этой группы можно усреднить оценки на просмотренные фильмы и для тех членов группы, у которых ещё не было просмотров этих фильмов экстраполировать значения оценок в этих ячейках.

Таким образом, у нас появляется некая средняя оценка в группе для каждого фильма из просмотренных, и мы можем предположить, что тем людям, которые ещё не успели посмотреть эти фильмы, они понравятся.
_________________________________________________________________________

Так как размерность по пользователям больше, чем размерность по фильмам, то выгоднее использовать не user-based алгоритм, а item-based. В этом случае вектор будет состоять не из оценок одного пользователя на различные фильмы, а будет содержать все оценки фильма от многих пользователей. Таким образом мы получим больший вектор, но само количество векторов будет меньше. А если меньше количество векторов, то проще посчитать матрицу из взаимной дистанции.

In [139]:
from surprise import KNNBasic

sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)#Инициируем алгоритм
trainingSet = data.build_full_trainset()#формирует датасет для тренировки
# После этого проводим тренировку модели на сформированном тренировочном датасете:
knn.fit(trainingSet)
# С помощью натренированной модели мы можем проскорить остальные оценки. 
# Для этого сгенерируем тестовый сет и построим предсказание по этому датасету:
testSet = trainingSet.build_anti_testset()
predictions = knn.test(testSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [141]:
testSet

[(608309, 30, 3.5888606677924892),
 (608309, 33, 3.5888606677924892),
 (608309, 42, 3.5888606677924892),
 (608309, 43, 3.5888606677924892),
 (608309, 44, 3.5888606677924892),
 (608309, 45, 3.5888606677924892),
 (608309, 46, 3.5888606677924892),
 (608309, 50, 3.5888606677924892),
 (608309, 77, 3.5888606677924892),
 (608309, 83, 3.5888606677924892),
 (608309, 84, 3.5888606677924892),
 (608309, 118, 3.5888606677924892),
 (608309, 125, 3.5888606677924892),
 (608309, 126, 3.5888606677924892),
 (608309, 143, 3.5888606677924892),
 (608309, 152, 3.5888606677924892),
 (608309, 166, 3.5888606677924892),
 (608309, 167, 3.5888606677924892),
 (608309, 175, 3.5888606677924892),
 (608309, 187, 3.5888606677924892),
 (608309, 188, 3.5888606677924892),
 (608309, 189, 3.5888606677924892),
 (608309, 197, 3.5888606677924892),
 (608309, 199, 3.5888606677924892),
 (608309, 210, 3.5888606677924892),
 (608309, 213, 3.5888606677924892),
 (608309, 223, 3.5888606677924892),
 (608309, 240, 3.5888606677924892),
 (6

In [187]:
predictions

[Prediction(uid=608309, iid=30, r_ui=3.5888606677924892, est=3.6247821794661266, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=608309, iid=33, r_ui=3.5888606677924892, est=3.075928845476297, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=608309, iid=42, r_ui=3.5888606677924892, est=2.5498296465626495, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=608309, iid=43, r_ui=3.5888606677924892, est=2.6713967891898616, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=608309, iid=44, r_ui=3.5888606677924892, est=3.4505711591041033, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=608309, iid=45, r_ui=3.5888606677924892, est=3.0744457347824277, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=608309, iid=46, r_ui=3.5888606677924892, est=3.4267973284074595, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=608309, iid=50, r_ui=3.5888606677924892, est=2.6736571

In [23]:
from collections import defaultdict
 
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

top3_recommendations = get_top3_recommendations(predictions)


In [143]:
def print_recs(i):
    for (a, b) in top3_recommendations[i]:
        print(titles[titles.Movie_Id == a]['Name'].values[0], np.round(b,2))

In [144]:
# выведем рекомендации для случайного пользователя
i = np.random.choice(list(top3_recommendations.keys()))

print_recs(i)

61* 4.5
It's a Mad 4.48
Field of Dreams 4.43


In [145]:
#Давайте посмотрим, что смотрел этот человек, и выберем из нашего датасета те фильмы, которые этот человек оценил на 5. 
films = data.df[(data.df.Cust_Id == i) & (data.df.Rating == 5)]['Movie_Id'].values
titles[titles.Movie_Id.isin(films)]['Name'].values

array(['North by Northwest', 'Rabbit-Proof Fence', 'Jaws',
       'Eat Drink Man Woman', 'Secondhand Lions', 'A Beautiful Mind',
       'Trapped', 'Eternal Sunshine of the Spotless Mind',
       'Being John Malkovich', 'Super Size Me',
       'The Silence of the Lambs', 'Riding Giants', 'Whale Rider',
       "The Wizard of Oz: Collector's Edition",
       'Finding Nemo (Widescreen)', 'The Shipping News', 'Seabiscuit',
       'Cool Hand Luke', 'Trading Places', "A Bug's Life", 'Rear Window',
       "Ferris Bueller's Day Off", 'Blood Simple', 'Altered States',
       'The Life of David Gale', 'Catch Me If You Can',
       "The Butterfly Effect: Director's Cut", 'Deliverance',
       'Raiders of the Lost Ark', 'A Clockwork Orange', 'Taxi Driver',
       'Psycho', 'The Incredibles', 'Fahrenheit 9/11', 'Frequency',
       'Forrest Gump', 'Hotel Rwanda',
       'Indiana Jones and the Temple of Doom', 'The French Connection',
       'Lost in Translation', 'The Godfather', 'Spy Game',
       "

# SVD-разложение

In [146]:
#матрица 'пользователь-фильм'
user_movie_matrix = np.array(((1,5,0,5,4), (5,4,4,3,2), (0,4,0,0,5), (4,4,1,4,0), (0,4,3,5,0), (2,4,3,5,3)))
user_movie_matrix

array([[1, 5, 0, 5, 4],
       [5, 4, 4, 3, 2],
       [0, 4, 0, 0, 5],
       [4, 4, 1, 4, 0],
       [0, 4, 3, 5, 0],
       [2, 4, 3, 5, 3]])

Сингулярное разложение матрицы — это некое новое представление исходной матрицы, которое отображено в сингулярных числах и сингулярных векторах. 

Так выглядит формула сингулярного разложения:

$M = U\sigma V$, где $M$ — исходная матрица 'пользователь-фильм'; $\sigma$ — матрица, на диагонали которой лежат сингулярные числа, а вне диагонали нули; $U$ — матрица-представление пользователей; $V$ — матрица-представление фильмов. 

Выполним сингулярное разложение при помощи библиотеки scipy 
scipy.linalg.svd-факторизует матрицу a на две унитарные матрицы U и Vh и 1-D массив сингулярных значений (вещественных, неотрицательных) таких, что a == U @ S @ Vh, где S-подходящая по форме матрица нулей с главной диагональю s.

In [163]:
from scipy import linalg
U, s, V = linalg.svd(user_movie_matrix) 
print('Shapes:')
print('user_movie_matrix:',user_movie_matrix.shape,'U:',U.shape,'s:',s.shape,'V:',V.shape)
sigma = np.zeros((user_movie_matrix.shape[0],user_movie_matrix.shape[1]))
for i in range(min(user_movie_matrix.shape[0],user_movie_matrix.shape[1])):
    sigma[i, i] = s[i]
user_movie_matrix1 = np.dot(U, np.dot(sigma, V))
np.allclose(user_movie_matrix, user_movie_matrix1)

Shapes:
user_movie_matrix: (6, 5) U: (6, 6) s: (5,) V: (5, 5)


True

На этом этапе мы получили новое матричное представление пользователей и фильмов. Заметим, что размерность нашего разложения совпадает с размерностью исходной матрицы. 

На практике обычно используют сокращенное представление, так как SVD-алгоритм обладает **свойством уменьшения размерности без значительной потери информации**. На больших матрицах 1 000 000 x 1 000 000 это особенно ощутимо. 

Чтобы сократить размерность, например до двух компонент, необходимо взять по две первых строки из каждой матрицы-представления. 

**Извлечем из матричных представлений первые две строки и сохраним их. Все дальнейшие действия будут производиться с ними.**

SVD-разложение на две компоненты произведено. Теперь посмотрим, какой фильм наше новое представление порекомендует в первую очередь для существующего пользователя.

Из исходной матрицы выберем вторую строчку (пользователь №2) — user_2. 3я строка.

Чтобы перевести пользователя в новое представление сниженной размерности, необходимо его исходный вектор умножить на транспонированную матрицу-представление фильмов.

$lowdim = user\_2 V^{T}$

In [166]:
user_movie_matrix[2]

array([0, 4, 0, 0, 5])

In [153]:
s

array([16.46644354,  6.21001334,  4.39908461,  2.90336429,  1.58445634])

In [179]:
V[:2]

array([[-0.31869431, -0.61196286, -0.29030808, -0.57523564, -0.32978869],
       [-0.40869555,  0.22187036, -0.37571481, -0.25558153,  0.75977493]])

In [180]:
lowdim = user_movie_matrix[2]@V[:2].T
lowdim

array([-4.09679486,  4.68635611])

**Финальный шаг.**
Необходимо произвести обратную трансформацию вектора в вектор оценок фильмов (т. е. в исходное представление фильмов). Для этого необходимо представление пользователя сниженной размерности умножить на матрицу-представление фильмов: 
$$inversed\_transformation=lowdimV$$

In [181]:
inversed_transformation = lowdim@V[:2]
inversed_transformation

array([-0.6096677 ,  3.54684982, -0.57140075,  1.15887633,  4.9116525 ])

In [183]:
user_movie_matrix[2][4]-inversed_transformation[4]

0.08834750418593273

In [184]:
new_user = np.array([0, 0, 3, 4, 0])
(new_user@V[:3].T)@V[:3]


array([ 0.40015936,  1.52990144,  1.47154395,  3.57090597, -1.13168846])

# Рекомендательная система на основе SVD разложения

In [188]:
# Возьмём подвыборку из 10000 случайных кастомеров и 5000 фильмов. 
cust_sample = df.Cust_Id.sample(10000)
movie_sample = df.Movie_Id.sample(5000)

#Возьмем только те оценки, которые относятся к выбранному подмножеству кастомеров и только те оценки, 
# которые относятся к выбранному подмножеству фильмов. Именно в такой последовательности — сначала Cust_Id, 
# затем Movie_Id, затем Rating. 

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[df.Cust_Id.isin(cust_sample) &
                              df.Movie_Id.isin(movie_sample)][['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [190]:
from surprise import SVD
svd = SVD()
trainingSet=data.build_full_trainset()#  формируем обучающую выборку
svd.fit(trainingSet)# тренировка алгоритма
testSet=trainingSet.build_testset()
predictions=svd.test(testSet)
top3_recommendations = get_top3_recommendations(predictions)
top3_recommendations

defaultdict(list,
            {1912665: [(3999, 4.807354712560082),
              (9929, 4.754032127273429),
              (5092, 4.71489411750821)],
             2460625: [(14550, 4.776118888895085),
              (3962, 4.640693398172551),
              (2452, 4.584307860062255)],
             306466: [(12293, 4.8333142761170675),
              (10042, 4.730669131981659),
              (3290, 4.658698095682554)],
             1499761: [(8524, 4.958581300448557),
              (571, 4.907505679669592),
              (12870, 4.7833912789108375)],
             1148143: [(16147, 4.873491199161442),
              (5760, 4.786836201309177),
              (15296, 4.778142491005903)],
             235838: [(6974, 5.0),
              (3446, 4.940513821589724),
              (7424, 4.916589614479708)],
             2091010: [(5549, 5.0), (12293, 5.0), (14550, 5.0)],
             1998117: [(10276, 4.527007710962749),
              (10729, 4.5090119175750445),
              (5515, 4.438923764297

In [191]:
# выведем рекомендации для случайного пользователя
i = np.random.choice(list(top3_recommendations.keys()))

print_recs(i)

You Can Count on Me 4.57
Spirited Away 4.52
Rushmore 4.41


In [192]:
#Давайте посмотрим, что смотрел этот человек, и выберем из нашего датасета те фильмы, которые этот человек оценил на 5. 
films = data.df[(data.df.Cust_Id == i) & (data.df.Rating == 5)]['Movie_Id'].values
titles[titles.Movie_Id.isin(films)]['Name'].values

array(['Boyz N the Hood', "Breakfast at Tiffany's", 'My Cousin Vinny',
       'Cool Hand Luke', 'Sling Blade', 'Memento', 'Lawrence of Arabia',
       'The Usual Suspects', 'The Princess Bride',
       "Gone with the Wind: Collector's Edition", 'The Fog of War',
       '12 Angry Men', 'Pi: Faith in Chaos', 'The Thin Red Line',
       'Pulp Fiction', 'The Godfather', 'Coming to America',
       "Boys Don't Cry", 'Mystic River', 'You Can Count on Me',
       'Three Kings', 'Traffic', 'The Big Lebowski',
       'Waiting for Guffman', 'Vertigo', 'Crouching Tiger', 'Magnolia'],
      dtype=object)

# Задача

In [4]:
# Загрузим датасет ml-100k:
data = surprise.Dataset.load_builtin("ml-100k")
trainingSet = data.build_full_trainset()
testSet = trainingSet.build_testset()



In [5]:
trainingSet.global_mean

3.52986

In [48]:
from surprise import KNNBasic

sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic()#Инициируем алгоритм
# После этого проводим тренировку модели на сформированном тренировочном датасете:
knn.fit(trainingSet)
# С помощью натренированной модели мы можем проскорить остальные оценки. 
# Для этого сгенерируем тестовый сет и построим предсказание по этому датасету:
predictions = knn.test(testSet)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [25]:
predictions[8]

Prediction(uid='196', iid='663', r_ui=5.0, est=3.6206349533418014, details={'actual_k': 39, 'was_impossible': False})

In [49]:
surprise.accuracy.rmse(predictions)

RMSE: 0.7781


0.7780707712292397

In [47]:
from surprise import SVD
random_state=829
data = surprise.Dataset.load_builtin("ml-100k")
trainingSet = data.build_full_trainset()
testSet = trainingSet.build_testset()
svd = SVD(random_state=829)
svd.fit(trainingSet)# тренировка алгоритма
predictionss=svd.test(testSet)
print(predictions[42][2]-predictions[42][3])
print(surprise.accuracy.rmse(predictionss))
predictions[42]

0.4241160916018458
RMSE: 0.6747
0.6746987152322735


Prediction(uid='186', iid='148', r_ui=4.0, est=3.575883908398154, details={'was_impossible': False})

In [30]:
# Cформируем датафрейм из нашего датасета.
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 3]
df.head()

,user_id,movie_id,rating,3
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [32]:
# Сформируйте для каждого user_id текстовую строку, содержащую набор movie_id, получивших оценку  и выше. Используйте пробел (" ") в качестве разделителя.
random_state=829
good = df[df['rating']>=4].groupby('user_id')['movie_id'].apply(lambda r: ' '.join([str(A) for A in r]))
good.head()

user_id
1      61 33 160 20 202 171 265 47 222 253 113 227 90...
10     16 486 611 7 100 488 285 504 289 340 505 489 6...
100    344 355 750 302 691 316 752 313 879 300 328 12...
101    405 121 24 284 50 237 181 924 257 742 255 117 ...
102    195 307 89 202 186 183 98 208 510 50 101 588 1...
Name: movie_id, dtype: object

In [34]:
good[2]

'344 355 750 302 691 316 752 313 879 300 328 1235 690 347 269 258 900 294 272 751 898 315'

In [35]:
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.2, 
                                   min_confidence=0.3, min_lift=2, 
                                   min_length=2)
association_rules

<generator object apriori at 0x0000025AEB648A50>

In [36]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]
asr_df

,from,to,confidence,support,lift
0,172,174,0.819113,0.254777,2.21725
1,173,174,0.762097,0.200637,2.06292
2,174,195,0.548851,0.20276,2.38257
3,174,204,0.543103,0.200637,2.17704
4,174,210,0.563218,0.208068,2.25767
5,174,79,0.591954,0.218684,2.1122
6,172,181 174,0.651877,0.20276,2.6583
7,172,174 50,0.78157,0.2431,2.51276
8,172,181 50,0.750853,0.233546,2.02666
9,174,98 50,0.577586,0.213376,2.12534


In [38]:
asr_df['from'].value_counts()

174    5
172    4
173    1
Name: from, dtype: int64